In [1]:
from tqdm.auto import tqdm

from datasets import load_dataset
from transformers import LlamaTokenizer, LlamaForCausalLM, AutoModelForCausalLM, get_scheduler

import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW

In [2]:
data_files = {"train": "data/ratings_train.csv", "test": "data/ratings_test.csv"}
dataset = load_dataset("csv", data_files= data_files)

Found cached dataset csv (/home/yehoon/.cache/huggingface/datasets/csv/default-a8458de001e53873/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-13b-hf")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [5]:
# def tokenize_function(examples, add_eos_token=True):
#     CUTOFF_LEN = 32000
#     result = tokenizer(
#         examples["document"],
#         truncation=True,
#         max_length = CUTOFF_LEN,
#         padding=False,
#         return_tensors=None,
#     )
# #     tokenizer.pad_token = tokenizer.eos_token
#     tokenizer.pad_token_id = (0)  # unk. we want this to be different from the eos token
#     tokenizer.padding_side = "left"
# #     tokenizer.add_special_tokens({'pad_token': '[PAD]'})
# #     return tokenizer((examples["document"]), padding="max_length", return_tensors="pt")
#     if (
#         result["input_ids"][-1] != tokenizer.eos_token_id
#         and len(result["input_ids"]) < CUTOFF_LEN
#         and add_eos_token
#     ):
#         result["input_ids"].append(tokenizer.eos_token_id)
#         result["attention_mask"].append(1)
 
#     result["labels"] = result["input_ids"].copy()
 
#     return result
# #     return tokenizer(examples["document"], padding=False, truncation=True, max_length=32000, return_tensors="pt")
# #     print(examples["document"])
# #     return tokenizer(examples["document"])

In [6]:
def tokenize_function(examples):
#     tokenizer.pad_token = tokenizer.eos_token
#     tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    tokenizer.pad_token_id = (0)
#     return tokenizer((examples["document"]), padding="max_length", return_tensors="pt")
    return tokenizer(examples["document"], padding="max_length", truncation=True, max_length=32000, return_tensors="pt")
#     print(examples["document"])
#     return tokenizer(examples["document"])

In [7]:
tokenized_dataset = dataset.map(tokenize_function)

Loading cached processed dataset at /home/yehoon/.cache/huggingface/datasets/csv/default-a8458de001e53873/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-e25f9018438636be.arrow


Map:   0%|          | 0/49997 [00:00<?, ? examples/s]

In [8]:
tokenizer.pad_token_id

0

In [9]:
# tokenized_dataset = tokenized_dataset.remove_columns(["document"])
# tokenized_dataset = tokenized_dataset.remove_columns(["label"])
# tokenized_dataset = tokenized_dataset.remove_columns(["attention_mask"])
tokenized_dataset.set_format("torch")

In [10]:
type(tokenized_dataset["train"]["input_ids"]), len(tokenized_dataset["train"]["input_ids"][0])

(torch.Tensor, 1)

In [ ]:
tokenized_dataset["train"]["input_ids"][0][0]

In [11]:
train_loader = DataLoader(tokenized_dataset["train"], shuffle=True, batch_size=256)
eval_loader = DataLoader(tokenized_dataset["test"], shuffle=True, batch_size=256)

In [12]:
model = LlamaForCausalLM.from_pretrained("decapoda-research/llama-7b-hf", torch_dtype=torch.float16, low_cpu_mem_usage=True)

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [13]:
optimizer = AdamW(model.parameters(), lr=5e-5)

In [14]:
num_epochs = 3
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [15]:
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device="cpu"
model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=31999)
    (layers): ModuleList(
      (0): LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
      (1): LlamaDecoderLayer(
  

In [16]:
model.

SyntaxError: invalid syntax (1068564517.py, line 1)

In [ ]:
# batch["input_ids"].unsqueeze(1)

In [ ]:
progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_loader:
#         print(batch)
#         batch = batch["input_ids"]
        print(batch)
        batch = {k: v for k, v in batch}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)